In [37]:
import json
import math

from numpy.random import choice, seed
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F

from models import BasicTransformer

## Data Preparation

In [67]:
import pandas as pd
from preprocess_data import *

with open('action_types.json', 'r') as f:
    action_types = json.load(f)
    
df = (
    pd.read_csv("WSL_actions.csv", index_col = 0)
    .pipe(add_coordinate_bins, n_bins_x = 10, n_bins_y = 10)
    .pipe(add_team_as_dummy)
    .pipe(get_action_type_names, action_types)
    .pipe(get_action_tokens)
    .assign(
        group_id = lambda d: d.groupby(['game_id', 'period_id']).ngroup(),
        action_token = lambda d: pd.Categorical(d.action_token)
    )
    [['group_id', 'action_token']]
)

vocab = df['action_token'].cat.categories

x, y = sequence_to_sliding_window(df['action_token'], n_prev_actions = 5)
y_list = list(y)
y = np.array([np.concatenate((x[i][1:], np.array([y_list[i]]))) for i in range(len(x))])

x_tok = np.zeros(x.shape, dtype=int)
y_tok = np.zeros(y.shape, dtype=int)

for i in range(x.shape[0]):
    for j in range(x.shape[1]):
        x_tok[i, j] = vocab.get_loc(x[i, j])

for i in range(y.shape[0]):
    for j in range(y.shape[1]):
        y_tok[i, j] = vocab.get_loc(y[i, j])



In [ ]:
train_groups = choice(df['group_id'].unique(), int(0.8 * df['group_id'].nunique()), replace = False)
train_groups[:5]

In [74]:
x_train, x_test, y_train, y_test = train_test_split(x_tok, y_tok, test_size=0.7, random_state=2024) 
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.7, random_state=2024) 

x_train = torch.Tensor(x_train).to(torch.int64)
x_val = torch.Tensor(x_val).to(torch.int64)
x_test = torch.Tensor(x_test).to(torch.int64)
y_train = torch.Tensor(y_train).to(torch.int64)
y_val = torch.Tensor(y_val).to(torch.int64)
y_test = torch.Tensor(y_test).to(torch.int64)

train_loader = torch.utils.data.DataLoader(
    list(zip(x_train, y_train)),
    batch_size=32,
    shuffle=True
)

val_loader = torch.utils.data.DataLoader(
    list(zip(x_val, y_val)),
    batch_size=32,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    list(zip(x_test, y_test)),
    batch_size=32,
    shuffle=True
)

In [53]:
def accuracy(preds, labels):
    return sum(preds[:, -1].argmax(dim=1) == labels[:, -1].argmax(dim=1)) / len(preds)

## Training Loop

In [77]:
def train(model, n_epochs=10):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    for epoch in range(n_epochs):
        model.train()
        i = 0
        train_loss = 0
        train_acc = 0
        for x_bat, y_bat in iter(train_loader):
            x_bat.permute(1, 0)
            y_bat = F.one_hot(y_bat, num_classes=len(vocab)).float()
            optimizer.zero_grad()
            y_pred = model(x_bat)
            loss = criterion(y_pred, y_bat)
            loss.backward()
            train_loss += loss.item()
            optimizer.step()
            i += 1
            train_acc += accuracy(y_pred, y_bat)
        
            # print(f'Epoch {epoch}, iter {i}, loss: {loss.item()}')
        train_acc = train_acc / len(train_loader)

        val_loss = 0
        val_acc = 0
        for x_val, y_val in iter(val_loader):
            y_pred = model(x_val)
            y_val = F.one_hot(y_val, num_classes=len(vocab)).float()
            loss = criterion(y_pred, y_val)
            val_loss += loss.item()
            val_acc += accuracy(y_pred, y_val)
        
        val_acc = val_acc / len(val_loader)
        print(f'Epoch {epoch}, iter {i}, train_loss: {train_loss}, train_acc: {train_acc}, val_loss: {val_loss}, val_acc: {val_acc}')

In [56]:
model = BasicTransformer(len(vocab), 128, 128, 2, 2, 0.1)

/Users/danielhocevar/miniconda3/envs/csc413/lib/python3.8/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [78]:
train(model)

KeyboardInterrupt: 